In [ ]:
import datetime
import pandas as pd
import os
import shutil

from enum import Enum
from datetime import datetime
from dateutil import parser
from pprint import pprint

In [3]:
# Documentation: serpapi.com/google-finance-api
# Installation: pip install google-search-results
from serpapi import GoogleSearch

def query_serpapi(ticker: str):
    params = {
        "api_key": os.getenv("GOOGLE_SEARCH_API_KEY"),
        "engine": "google_finance",
        "hl": "en",
        "q": f"{ticker}",
        "window": "YTD"
    }
    
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

In [ ]:
class Country(Enum):
    GERMANY = 1
    JAPAN = 2
    UNITED_STATES = 3
    
METAPLANET_TICKERS = {
    Country.JAPAN: "3350:TYO",
    Country.GERMANY: "DN3:FRA",
    Country.UNITED_STATES: "MTPLF:OTCMKTS"
}

responses = {}
for country in Country:
    print(METAPLANET_TICKERS[country])
    responses[country] = query_serpapi(METAPLANET_TICKERS[country])

In [ ]:
rows = {}

for country, response in responses.items():
    rows[country] = {}

    print(f"Processing Google Finance API Results for {country.name}")
    
    # DEBUGGING purposes
    # pprint(response)
    
    for daily_summary in response['graph']:
        ticker = METAPLANET_TICKERS[country].split(":")[0]
        currency = daily_summary['currency'] if 'currency' in daily_summary else None
        date = daily_summary['date'] if 'date' in daily_summary else None
        price = daily_summary['price'] if 'price' in daily_summary else None
        volume = daily_summary['volume'] if 'volume' in daily_summary else None

        if (currency is None
            or date is None
            or price is None
            or volume is None):
            print(f"ERROR processing row {daily_summary}. One of the [currency, date, price, volume] is missing.")
        
        trading_day = parser.parse(date).strftime('%Y-%m-%d')

        # rows.append([trading_day, ticker, price, currency, volume])
        # If this date is not yet in rows, or this row has volume and the existing one doesn't
        # Some of the rows returned by the API is scuffed.
        row = [trading_day, ticker, price, currency, volume]
        if (trading_day not in rows[country]) or (row[4] is not None and rows[country][trading_day][4] is None):
            rows[country][trading_day] = row

rows = [row for country_rows in rows.values() for row in country_rows.values()]
pprint(rows)

In [ ]:
schema = [
    "day",
    "ticker",
    "price",
    "currency",
    "volume"
]

pd.set_option('display.max_rows', None)
df = pd.DataFrame(rows, columns=schema)

# Trading order: JP -> DE -> US
ticker_order = ["3350", "DN3", "MTPLF"]
df['ticker'] = pd.Categorical(df['ticker'], categories=ticker_order, ordered=True)
df = df.sort_values(by=['day', 'ticker'], ascending=[False, False])
df

In [ ]:
DIRECTORY = os.path.join(os.getcwd(), 'dataset')
os.makedirs(DIRECTORY, exist_ok=True)
df.to_csv(os.path.join(DIRECTORY, 'metaplanet_daily_price_and_volume.csv'), index=False)

# Move the updated CSV for upload
try:
    shutil.copy(csv_path, DIRECTORY)
except Exception as e:
    print(f"Exception: {e}")

# Update csv file using Kaggle API
execution_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
message = f"Added new daily price/vol data from Google Finance API {execution_date}"

os.system(f'kaggle datasets version -p {DIRECTORY} -m "{message}"')